In [1]:
# here I am attempting to solve the first scenario of the requirements
# most of the stuff to solve this is done in the labs
# Scenario 1 (real world) - you receive a test set containing 55 scanned images annotated with the option (F or I) and with the digit (1, 2, 3 or 4). For each image you have
# to output the corresponding grade. Each correctly labeled scanned image will worth
# 0.03 points for a total of 1.65 points;

In [2]:
# Basically, I will do the following steps:
# 1. Apply filters on images and detect the edges (canny edge?)
#    1.a grayscale the image
#    1.b apply filter (gaussian, sobel?)
#    1.c canny edge detection?
# 2. Detect lines
#    2.a Detect horizontal lines
#    2.b Detect vertical lines
#    2.c Merge image
#3. Detect boxes and create an inside, smaller box, as to not count the edges
#    3.a Detect boxes (lab 3.1)
#    3.b Verify mean color inside the boxes and mark as green with X and red without X
#    3.c Store the data and compare it to the answers in the files
#    3.d Do this on all files and see accuracy

In [3]:
# imports
# Required libraries to be installed: Pillow, OpenCV
import cv2 as cv
import numpy as np
import glob
import os
import pickle
import matplotlib.pyplot as plt
# from google.colab.patches import cv2_imshow
from IPython.display import clear_output, Image, display
import PIL.Image
import io

In [4]:
base_folder = './Files/images/' # change this on your machine
images = glob.glob(os.path.join(base_folder, "image_*.jpg")) 
char_to_index = {'A': 0, 'B': 1, 'C': 2, 'D': 3}
image_index = 1

In [5]:
# global variables

# kernel size. h and v stand for orientation
# hk_size = 3
# vk_size = 3

In [329]:
# show image function is taken from Andrei Manolache outside of the project scope, I trust it does not count as plagiarism 
def show_image(a, fmt='jpeg'):
    a = np.uint8(np.clip(a, 0, 255))
    f = io.BytesIO()
    PIL.Image.fromarray(a).save(f, fmt)
    display(Image(data=f.getvalue()))
    
def read_image(index):
    image = cv.imread(images[index])
    return cv.resize(image, (0, 0), fx=0.15, fy=0.15)

def split_first_table(image):
    image_h, image_w, channels = image.shape
    return image[int(0.45 * image_h):int(image_h * 0.88), int(image_w * 0.1): int(image_w * 0.4)]

def split_second_table(image):
    image_h, image_w, channels = image.shape
    return image[int(0.45 * image_h):int(image_h * 0.88), int(image_w * 0.60): int(image_w * 0.90)]

def apply_filters(image):
    image_gray = cv.cvtColor(image, cv.COLOR_BGR2GRAY)
    return image_gray

def get_binary_image(image_gray):
    mean_value = image_gray.mean()
    _, image_th = cv.threshold(imgage_gray,meanValue, 255, cv.THRESH_BINARY_INV)

def horizontal_kernel(hk_size=11):
    kernel = np.array([hk_size * [0], hk_size * [1], hk_size * [0]])
    kernel = kernel / kernel.sum()
    return kernel

def vertical_kernel(vk_size=11):
#     kernel = np.array(vk_size * [int(vk_size / 2) * [0] + [1] + int(vk_size / 2) * [0]])
    kernel = np.array([[0,1,0],[0,1,0],[0,1,0]])                
    kernel = kernel / kernel.sum()
    return kernel

def filter_vertical(image_gray):    
    mean_value = image_gray.mean()
    _, image_th = cv.threshold(image_gray, mean_value, 255, cv.THRESH_BINARY_INV)
    thresholded = image_th
    filtered = 255 - cv.filter2D(255 - thresholded, -1, vertical_kernel())
    filtered[filtered < 255] = 0
    filtered_vertical = filtered
    return filtered_vertical


def filter_horizontal(image_gray):    
    mean_value = image_gray.mean()
    _, image_th = cv.threshold(image_gray, mean_value, 255, cv.THRESH_BINARY_INV)
    thresholded = image_th
    filtered = 255 - cv.filter2D(255 - thresholded, -1, horizontal_kernel())
    filtered[filtered < 255] = 0
    filtered_horizontal = filtered
    return filtered_horizontal

def get_horizontal_lines(image_gray, filtered_horizontal, num_lines):
    mask = filtered_horizontal // 255
    res = np.sum(mask, axis=1) 
    res = res.argsort() 
    h_img = np.dstack((filtered_horizontal, filtered_horizontal, filtered_horizontal))
    h_lines = [] #  _ x 
    for i in range(num_lines+1):
        cv.line(h_img, (0, res[-i]), (h_img.shape[1], res[-i]), (0, 0, 255), 2) 
        h_lines.append([(0, res[-i]), (h_img.shape[1], res[-i])])
    
    h_lines.sort(key=lambda coords: coords[0][1])
    
    #### h lines
    threshold_same_h_line = 15
    distinct_h_lines = []   
    distinct_h_lines.append(h_lines[0])

    for line in h_lines:  
        if line[0][1] - distinct_h_lines[-1][0][1] > threshold_same_h_line:
            distinct_h_lines.append(line)   

    # take the last 16 lines
    correct_h_lines = distinct_h_lines[-16:] 
    
    return correct_h_lines

def get_vertical_lines(image_gray, filtered_vertical, num_lines):
    mask = filtered_vertical // 255
    res = np.sum(mask, axis=0) 
    res = res.argsort() 
    v_img = np.dstack((filtered_vertical, filtered_vertical, filtered_vertical))
    v_lines = [] #  _ x 
    for i in range(num_lines+1):
        cv.line(v_img, (res[-i], 0), (res[-i], v_img.shape[0]), (0, 0, 255), 1) 
        v_lines.append([(res[-i], 0), (res[-i], v_img.shape[0])])
    # de aici
    v_lines.sort(key=lambda coords: coords[1][0])
    #### v lines
    threshold_same_v_line = 20
    distinct_v_lines = []   
    distinct_v_lines.append(v_lines[0])

    for line in v_lines:  
        if line[1][0] - distinct_v_lines[-1][1][0] > threshold_same_v_line:
            distinct_v_lines.append(line)   

    # take the last 16 lines
    correct_v_lines = distinct_v_lines[-5:] 
    
    return correct_v_lines

def apply_lines(image):
    image_gray = apply_filters(image)
    image_h = filter_horizontal(image_gray)
    image_v = filter_vertical(image_gray)
    h_lines = get_horizontal_lines(image_gray, image_h, 60)
    v_lines = get_vertical_lines(image_gray, image_v, 60)
    
    color_image = np.dstack((image_gray, image_gray, image_gray))
    for line in v_lines: 
        cv.line(color_image, line[0], line[1], (255, 0, 0), 1) 
        
    for line in h_lines: 
        cv.line(color_image, line[0], line[1], (0, 0, 255), 1)     
    ###
    return color_image

def find_x_from_img(grayscale_image, vertical_lines, horizontal_lines, threshold):
    # grayscale_image - input image containing the frame
    # vertical_lines - list with the vertical lines
    # horizontal_lines - list with horizontal lines
    # threshold - simple 1D classifier    
    appended = False
    image = np.dstack((grayscale_image, grayscale_image, grayscale_image))
    x_color = (0, 255, 0)  # plot a patch containing an X with green color
    blank_color = (0, 0, 255)  # plot a patch containing a blank with red color
    # crop each patch and display it
    x_values = [] 
    for i in range(len(horizontal_lines) - 1):
        appended = False
        for j in range(len(vertical_lines) - 1):
            x_min = vertical_lines[j][0][0] + 3
            x_max = vertical_lines[j + 1][1][0] - 2
            y_min = horizontal_lines[i][0][1] + 3
            y_max = horizontal_lines[i + 1][1][1] - 2
        
            
            patch = grayscale_image[y_min:y_max,x_min:x_max].copy()
            mean_patch_value = np.round(patch.mean())
            if(mean_patch_value <= threshold):
              color = x_color
              x_values.append(j)
              appended = True
            else:
              color = blank_color
            cv.rectangle(image, (x_min, y_min), (x_max, y_max), color=color, thickness=1)
        if (appended == False):
            x_values.append(0)
#             cv.putText(image, str(mean_patch_value)[:3] ,(x_min + 10, y_min + 50), cv.FONT_HERSHEY_COMPLEX, 1, (0,0,0), 2) 
#     show_image(image)
    return x_values

def verify_result(ground_truth, detected_values):
    for i in range(0, 15):
        if (int(char_to_index[ground_truth[i][1]]) != detected_values[i]):
            return 0
    return 1

In [ ]:
image_matching = np.empty((150,3))
image_matching[:] = np.NaN
x_threshold = 245
for i in range(0, 150):
    # read image and apply lines on each table
    image = read_image(i)
    image_right = split_second_table(image)
    
    image_name = images[i].split('/')[-1].split('.')[-2]
    ground_truth_content = np.loadtxt(os.path.join(base_folder, '%s.txt' % image_name), dtype=str)
#     print (image_name)
    #left
    image_left = split_first_table(image)
    image_lg = apply_filters(image_left)
    image_h = filter_horizontal(image_lg)
    image_v = filter_vertical(image_lg)
    h_lines_l = get_horizontal_lines(image_lg, image_h, 60)
    v_lines_l = get_vertical_lines(image_lg, image_v, 60)
    
    ground_truth_left = ground_truth_content[1:16]
    
    #right
    image_right = split_second_table(image)
    image_rg = apply_filters(image_right)
    image_h = filter_horizontal(image_rg)
    image_v = filter_vertical(image_rg)
    h_lines_r = get_horizontal_lines(image_rg, image_h, 60)
    v_lines_r = get_vertical_lines(image_rg, image_v, 60)
    
    ground_truth_right = ground_truth_content[16:-1]
    
    results_left = find_x_from_img(image_lg, v_lines_l, h_lines_l, x_threshold)
    results_right = find_x_from_img(image_rg, v_lines_r, h_lines_r, x_threshold)
    image_matching[i][0] = verify_result(ground_truth_left, results_left)
    image_matching[i][1] = verify_result(ground_truth_right, results_right)
    image_matching[i][2] = (image_matching[i][0] + image_matching[i][1]) // 2
#     print (image_matching[i][0], image_matching[i][1], image_matching[i][2])
#     print (image_name, image_matching[i][0] == 1, image_matching[i][1] == 1, image_matching[i][2] == 1)

# print ("left:")
# print (image_matching)
total = np.sum(image_matching, axis=0)
print(total[0] * 100 / 150, total[1] * 100 / 150, total[2] * 100 / 150)

In [ ]:
# # testing flow
# # read image

# img = read_image(image_index)

# # cut image
# img = split_first_table(img)

# show_image(img)
# #apply filters
# img_gray = apply_filters(img)

# # obtain binary image
# filtered_h = filter_horizontal(img_gray)

# horizontal_l = get_horizontal_lines(filtered_h, 15)
# show_image(horizontal_l)

In [10]:
# # testing written functions
# img = read_image(image_index)
# img1 = split_first_table(img)
# img2 = split_second_table(img)
# # show_image(img1)
# # show_image(img2)
# img1_gray = apply_filters(img1)
# show_image(img1_gray)
# img1_filtered_vertical = filter_vertical(img1_gray)
# show_image(img1_filtered_vertical)
# vertical_l = get_vertical_lines(img1_filtered_vertical, 15)
# show_image(vertical_l)

In [11]:
len(images)

150